In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from matplotlib import animation
from IPython.display import HTML
import math

class DifferentialDriveRobot:
    def __init__(self, x=0, y=0, theta=0, wheel_radius=0.05, wheel_base=0.2, 
                 mass=1.0, moment_of_inertia=0.1, dt=0.1):
        # State variables
        self.x = x              # x-position [m]
        self.y = y              # y-position [m]
        self.theta = theta      # orientation [rad]
        
        # Robot parameters
        self.wheel_radius = wheel_radius  # wheel radius [m]
        self.wheel_base = wheel_base      # distance between wheels [m]
        self.mass = mass                  # robot mass [kg]
        self.moment_of_inertia = moment_of_inertia  # moment of inertia [kg*m^2]
        
        # Simulation parameters
        self.dt = dt            # time step [s]
        
        # Wheel velocities and torques
        self.omega_l = 0        # left wheel angular velocity [rad/s]
        self.omega_r = 0        # right wheel angular velocity [rad/s]
        self.tau_l = 0          # left wheel torque [N*m]
        self.tau_r = 0          # right wheel torque [N*m]
        
        # Dynamic model parameters (calculated from robot parameters)
        self.J_r = (self.mass * self.wheel_radius**2 / 4) + (self.moment_of_inertia * self.wheel_radius**2 / self.wheel_base**2)
        self.J_l = self.J_r
        self.J_rl = (self.mass * self.wheel_radius**2 / 4) - (self.moment_of_inertia * self.wheel_radius**2 / self.wheel_base**2)
        
        # Friction coefficients
        self.B_r = 0.01  # viscous friction coefficient for right wheel
        self.B_l = 0.01  # viscous friction coefficient for left wheel
        
        # History for plotting
        self.history_x = [self.x]
        self.history_y = [self.y]
        self.history_theta = [self.theta]
        self.history_v = [0]  # Linear velocity
        self.history_omega = [0]  # Angular velocity
        self.time = [0]
    
    def set_wheel_torques(self, tau_l, tau_r):
        """Set the wheel torques"""
        self.tau_l = tau_l
        self.tau_r = tau_r
    
    def forward_kinematics(self):
        """Calculate robot velocities from wheel velocities"""
        # Linear and angular velocity of the robot
        v = self.wheel_radius * (self.omega_r + self.omega_l) / 2
        omega = self.wheel_radius * (self.omega_r - self.omega_l) / self.wheel_base
        
        return v, omega
    
    def inverse_kinematics(self, v, omega):
        """Calculate wheel velocities from robot velocities"""
        # Angular velocities of the wheels
        omega_l = (2 * v - omega * self.wheel_base) / (2 * self.wheel_radius)
        omega_r = (2 * v + omega * self.wheel_base) / (2 * self.wheel_radius)
        
        return omega_l, omega_r
    
    def update_dynamics(self):
        """Update wheel velocities based on applied torques and dynamic model"""
        # Dynamic equations relating torques to wheel accelerations
        # [tau_r]   [J_r  J_rl] [omega_dot_r]
        # [tau_l] = [J_rl J_l ] [omega_dot_l]
        
        # Including friction: tau = J*omega_dot + B*omega
        
        # Compute the determinant of the inertia matrix
        det_J = self.J_r * self.J_l - self.J_rl**2
        
        # Compute the acceleration of the wheels
        omega_dot_r = ((self.J_l * (self.tau_r - self.B_r * self.omega_r) - 
                        self.J_rl * (self.tau_l - self.B_l * self.omega_l)) / det_J)
        
        omega_dot_l = ((self.J_r * (self.tau_l - self.B_l * self.omega_l) - 
                        self.J_rl * (self.tau_r - self.B_r * self.omega_r)) / det_J)
        
        # Update wheel velocities
        self.omega_r += omega_dot_r * self.dt
        self.omega_l += omega_dot_l * self.dt
    
    def update_kinematics(self):
        """Update robot state based on wheel velocities"""
        # Get robot velocities from wheel velocities
        v, omega = self.forward_kinematics()
        
        # Update robot position and orientation
        self.x += v * math.cos(self.theta) * self.dt
        self.y += v * math.sin(self.theta) * self.dt
        self.theta += omega * self.dt
        
        # Normalize theta to [-pi, pi]
        self.theta = math.atan2(math.sin(self.theta), math.cos(self.theta))
        
        # Update history
        self.history_x.append(self.x)
        self.history_y.append(self.y)
        self.history_theta.append(self.theta)
        self.history_v.append(v)
        self.history_omega.append(omega)
        self.time.append(self.time[-1] + self.dt)
    
    def update(self):
        """Complete update step: dynamics + kinematics"""
        self.update_dynamics()
        self.update_kinematics()
    
    def plot_path(self):
        """Plot the robot's path"""
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(self.history_x, self.history_y, 'b-')
        plt.plot(self.history_x[0], self.history_y[0], 'go', markersize=10)  # Start position
        plt.plot(self.history_x[-1], self.history_y[-1], 'ro', markersize=10)  # End position
        plt.grid(True)
        plt.axis('equal')
        plt.xlabel('X [m]')
        plt.ylabel('Y [m]')
        plt.title('Robot Path')
        
        plt.subplot(1, 2, 2)
        plt.plot(self.time, self.history_v, 'r-', label='Linear Velocity [m/s]')
        plt.plot(self.time, self.history_omega, 'g-', label='Angular Velocity [rad/s]')
        plt.grid(True)
        plt.xlabel('Time [s]')
        plt.ylabel('Velocity')
        plt.title('Robot Velocities')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

    def animate_robot(self, skip_frames=1):
        """Animate the robot's movement"""
        # Create figure and axis
        fig, ax = plt.subplots(figsize=(10, 8))
        ax.set_xlim([min(self.history_x) - 0.5, max(self.history_x) + 0.5])
        ax.set_ylim([min(self.history_y) - 0.5, max(self.history_y) + 0.5])
        ax.grid(True)
        ax.set_xlabel('X [m]')
        ax.set_ylabel('Y [m]')
        ax.set_title('Mobile Robot Simulation')
        
        # Initialize robot body (rectangle) and wheels (circles)
        robot_length = 0.3
        robot_width = 0.2
        
        # These will be updated in the animation
        robot_body = Rectangle((0, 0), robot_length, robot_width, color='blue', alpha=0.7)
        left_wheel = Circle((0, 0), self.wheel_radius, color='black')
        right_wheel = Circle((0, 0), self.wheel_radius, color='black')
        
        # Add patches to the axis
        ax.add_patch(robot_body)
        ax.add_patch(left_wheel)
        ax.add_patch(right_wheel)
        
        # Plot the full path
        path_line, = ax.plot([], [], 'b-', alpha=0.5)
        
        # Time text
        time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
        
        def init():
            robot_body.set_xy((0, 0))
            left_wheel.center = (0, 0)
            right_wheel.center = (0, 0)
            path_line.set_data([], [])
            time_text.set_text('')
            return robot_body, left_wheel, right_wheel, path_line, time_text
        
        def animate(i):
            i = i * skip_frames  # Skip frames to speed up animation
            if i >= len(self.history_x):
                i = len(self.history_x) - 1
                
            # Current position and orientation
            x, y, theta = self.history_x[i], self.history_y[i], self.history_theta[i]
            
            # Robot body position (centered at the midpoint between wheels)
            # We need to adjust the position as Rectangle is positioned by its bottom-left corner
            body_x = x - (robot_length/2) * math.cos(theta) - (robot_width/2) * math.sin(theta)
            body_y = y - (robot_length/2) * math.sin(theta) + (robot_width/2) * math.cos(theta)
            
            # Update robot body
            robot_body.set_xy((body_x, body_y))
            robot_body.angle = math.degrees(theta)
            
            # Wheel positions
            wheel_offset = self.wheel_base / 2
            left_x = x - wheel_offset * math.sin(theta)
            left_y = y + wheel_offset * math.cos(theta)
            right_x = x + wheel_offset * math.sin(theta)
            right_y = y - wheel_offset * math.cos(theta)
            
            # Update wheel positions
            left_wheel.center = (left_x, left_y)
            right_wheel.center = (right_x, right_y)
            
            # Update path
            path_line.set_data(self.history_x[:i+1], self.history_y[:i+1])
            
            # Update time text
            time_text.set_text(f'Time: {self.time[i]:.1f}s')
            
            return robot_body, left_wheel, right_wheel, path_line, time_text
        
        # Number of frames
        frames = len(self.history_x) // skip_frames
        
        # Create animation
        anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, 
                                      interval=50, blit=True)
        
        # Display the animation
        plt.close()  # To prevent displaying the figure twice
        return HTML(anim.to_jshtml())

# Example usage

# Simulation Parameters
sim_time = 10.0  # seconds
dt = 0.05        # time step (seconds)
num_steps = int(sim_time / dt)

# Initialize robot
robot = DifferentialDriveRobot(x=0, y=0, theta=0, wheel_radius=0.05, wheel_base=0.2, 
                             mass=1.0, moment_of_inertia=0.1, dt=dt)

# Run simulation
for t in range(num_steps):
    # Example 1: Constant torque (circular motion)
    if t < num_steps / 3:
        robot.set_wheel_torques(0.2, 0.3)
    # Example 2: Go straight
    elif t < 2 * num_steps / 3:
        robot.set_wheel_torques(0.3, 0.3)
    # Example 3: Turn right
    else:
        robot.set_wheel_torques(0.3, 0.1)
    
    # Update robot state
    robot.update()

# Plot the robot's path
robot.plot_path()

# Animate the robot
HTML(robot.animate_robot(skip_frames=2).data)

# Example of using inverse kinematics to follow a specific path

# Create a new robot
robot2 = DifferentialDriveRobot(x=0, y=0, theta=0, wheel_radius=0.05, wheel_base=0.2, 
                              mass=1.0, moment_of_inertia=0.1, dt=dt)

# Parameters for a figure-8 path
t_values = np.linspace(0, 2*np.pi, num_steps)
scale_x = 2.0
scale_y = 1.0

# Define the desired path (figure-8)
path_x = scale_x * np.sin(t_values)
path_y = scale_y * np.sin(t_values) * np.cos(t_values)

# For storing the target path
target_x = []
target_y = []

# PID controller parameters
kp_v = 2.0    # Proportional gain for velocity
kp_omega = 5.0  # Proportional gain for angular velocity

# Run simulation
for t in range(num_steps):
    # Current robot position
    current_x = robot2.x
    current_y = robot2.y
    current_theta = robot2.theta
    
    # Target position (current point on the path)
    target_x.append(path_x[t])
    target_y.append(path_y[t])
    
    # Calculate distance and heading to target
    dx = path_x[t] - current_x
    dy = path_y[t] - current_y
    distance = np.sqrt(dx**2 + dy**2)
    
    # Calculate target heading (direction to the target)
    if t < num_steps - 1:
        # Look ahead for smoothing
        target_heading = np.arctan2(path_y[t+1] - path_y[t], path_x[t+1] - path_x[t])
    else:
        target_heading = np.arctan2(dy, dx)
    
    # Calculate heading error (between current robot heading and target heading)
    heading_error = target_heading - current_theta
    
    # Normalize heading error to [-pi, pi]
    heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))
    
    # Calculate control inputs using PID
    v = kp_v * distance  # Linear velocity proportional to distance
    omega = kp_omega * heading_error  # Angular velocity proportional to heading error
    
    # Limit the control inputs
    v = np.clip(v, -1.0, 1.0)
    omega = np.clip(omega, -2.0, 2.0)
    
    # Convert to wheel velocities
    omega_l, omega_r = robot2.inverse_kinematics(v, omega)
    
    # Convert wheel velocities to torques (simplified: torque = desired angular velocity)
    tau_l = omega_l * 0.1  # Simple gain
    tau_r = omega_r * 0.1  # Simple gain
    
    # Apply torques
    robot2.set_wheel_torques(tau_l, tau_r)
    
    # Update robot state
    robot2.update()

# Plot the robot's path and target path
plt.figure(figsize=(12, 6))
plt.plot(robot2.history_x, robot2.history_y, 'b-', label='Robot Path')
plt.plot(target_x, target_y, 'r--', label='Target Path')
plt.plot(robot2.history_x[0], robot2.history_y[0], 'go', markersize=10)  # Start position
plt.plot(robot2.history_x[-1], robot2.history_y[-1], 'mo', markersize=10)  # End position
plt.grid(True)
plt.axis('equal')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('Path Following using Inverse Kinematics')
plt.legend()
plt.show()

# Animate the path following
HTML(robot2.animate_robot(skip_frames=2).data)

ModuleNotFoundError: No module named 'numpy'